# TFX Keras Component Tutorial to predict Sepsis from patient vitals data

A Component-by-Component Introduction to TensorFlow Extended (TFX)

Note: In this notebook, we can instantiate components one-by-one and run them with InteractiveContext.run(). By contrast, in a production setting, we would specify all the components upfront in a Pipeline to pass to the orchestrator (see the Building a TFX Pipeline Guide). https://www.tensorflow.org/tfx/guide/build_tfx_pipeline

In [1]:
import os
import pprint
import tempfile
import urllib
import tensorboard

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

# Check Versions
Expecting
- TensorFlow version: 2.8.0
- TFX version: 1.7.1

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.0
TFX version: 1.7.1


# Setup pipeline paths

In [3]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Vitals Pipeline example.
_vitals_root = os.path.join(_tfx_root, '../pipeline/vitals')

# This is the path where your model will be pushed for serving.
_serving_model_dir = '../models/vitals_simple'

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

# Fetch training data and attributes

In [4]:
_attr_root = '../data/attributes'
ATTR_PATH = 'https://raw.githubusercontent.com/redhat-na-ssa/mlops-prototype/main/data/dataSepsis/csv_format/attribute_definitions.csv'
_attr_filepath = os.path.join(_attr_root, "attr_definitions.csv")
urllib.request.urlretrieve(ATTR_PATH, _attr_filepath)

('../data/attributes/attr_definitions.csv',
 <http.client.HTTPMessage at 0x7f2a2ae5db20>)

In [5]:
_data_root = '../data/raw'
DATA_PATH = 'https://raw.githubusercontent.com/redhat-na-ssa/mlops-prototype/main/data/dataSepsis/csv_format/pat_vitals_labeled-dataSepsis.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('../data/raw/data.csv', <http.client.HTTPMessage at 0x7f2a2ae5d400>)

# View the data

In [6]:
# view the attributes about the vitals data
!head {_attr_filepath}

target_file,attribute_name,attribute_definition
,List all the attributes in the dataset. Label continuous attributes with c and discrete with d,
,,
pat_vitals_labeled.csv,Vital signs (columns 1-8),"Doctors order, basis every 4 hours, least invasive, cost least amount of money"
pat_vitals_labeled.csv,HR,Heart rate (beats per minute)
pat_vitals_labeled.csv,O2Sat,Pulse oximetry (%)
pat_vitals_labeled.csv,Temp,Temperature (Deg C)
pat_vitals_labeled.csv,SBP,Systolic BP (mm Hg)
pat_vitals_labeled.csv,MAP,Mean arterial pressure (mm Hg)
pat_vitals_labeled.csv,DBP,Diastolic BP (mm Hg)


In [7]:
# view the actual patient data with sepsis labels
# 1 = septic, 0 = not-septic
!head {_data_filepath}

patient_id,record_date,record_time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,isSepsis
1,,,63,90,40.3,NaN,NaN,NaN,17,NaN,0
2,,,79,95,39.2,143,77,47,13,NaN,0
3,,,87,94,40.3,133,74,48,20,NaN,0
4,,,71,100,42.1,NaN,NaN,NaN,15,NaN,0
5,,,68,94.5,39.7,147.5,102,NaN,20,NaN,0
6,,,78,99,39.6,100,67,49.5,18,NaN,0
7,,,242,NaN,39.30,NaN,NaN,NaN,33,NaN,1
8,,,81,100,40.3,112,79.5,63,18,NaN,0
9,,,178,100,39.22,141,85,57,22,NaN,1


# Create the Interactive Context

In [8]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext(pipeline_root='../pipeline')

# ExampleGen
- The ExampleGen component is usually at the start of a TFX pipeline. It will:
- Split data into training and evaluation sets (by default, 2/3 training + 1/3 eval)
- Convert data into the tf.Example format
- Copy data into the _tfx_root directory for other components to access ExampleGen takes as input the path to your data source. In our case, this is the _data_root path that contains the downloaded CSV.

## Enable Cache
When using the InteractiveContext in a notebook to develop a pipeline you can control when individual components will cache their outputs. 
1. Set enable_cache to True when you want to reuse the previous output artifacts that the component generated. 
1. Set enable_cache to False when you want to recompute the output artifacts for a component, if you are making changes to the code for example.

In [9]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ../data/raw/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Examine ExampleGen artifacts. 
This component produces two artifacts, training examples and evaluation examples: 'train' 'eval'

In [10]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] ../pipeline/CsvExampleGen/examples/1


In [11]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "DBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "EtCO2"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "HR"
    value {
      int64_list {
        value: 63
      }
    }
  }
  feature {
    key: "MAP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "O2Sat"
    value {
      float_list {
        value: 90.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      int64_list {
        value: 17
      }
    }
  }
  feature {
    key: "SBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 40.29999923706055
      }
    }
  }
  feature {
    key: "isSepsis"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "patient_id"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
 

# StatisticsGen

- The StatisticsGen component computes statistics over your dataset for data analysis, as well as for use in downstream components. 
- It uses the TensorFlow Data Validation library.
- StatisticsGen takes as input the dataset we just ingested using ExampleGen.

In [12]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to ../pipeline/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to ../pipeline/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

## Examine StatisticsGen artifacts.
We can visualize the outputted statistics. Try playing with the different plots!

In [13]:
context.show(statistics_gen.outputs['statistics'])

# SchemaGen

The SchemaGen component generates a schema based on your data statistics. 
- (A schema defines the expected bounds, types, and properties of the features in your dataset.) 
- It also uses the TensorFlow Data Validation library.
- Note: The generated schema is best-effort and only tries to infer basic properties of the data. It is expected that you review and modify it as needed.
- SchemaGen will take as input the statistics that we generated with StatisticsGen, looking at the training split by default.

In [14]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to ../pipeline/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

## Examine SchemaGen artifacts.

After SchemaGen finishes running, we can visualize the generated schema as a table.
- Each feature in your dataset shows up as a row in the schema table, alongside its properties. 
- The schema also captures all the values that a categorical feature takes on, denoted as its domain.

In [15]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'DBP',FLOAT,required,single,-
'EtCO2',FLOAT,required,single,-
'HR',INT,required,single,-
'MAP',FLOAT,required,single,-
'O2Sat',FLOAT,required,single,-
'Resp',INT,required,single,-
'SBP',FLOAT,required,single,-
'Temp',FLOAT,required,single,-
'isSepsis',INT,required,single,-


# ExampleValidator

The ExampleValidator component detects anomalies in your data, based on the expectations defined by the schema. 
- It also uses the TensorFlow Data Validation library.
- ExampleValidator will take as input the statistics from StatisticsGen, and the schema from SchemaGen.

In [16]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to ../pipeline/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to ../pipeline/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

## Examine ExampleValidator artifacts.
After ExampleValidator finishes running, we can visualize the anomalies as a table.

In the anomalies table, we can see that there are no anomalies. This is what we'd expect, since this the first dataset that we've analyzed and the schema is tailored to it. You should review this schema -- anything unexpected means an anomaly in the data. Once reviewed, the schema can be used to guard future data, and anomalies produced here can be used to debug model performance, understand how your data evolves over time, and identify data errors.

In [17]:
context.show(example_validator.outputs['anomalies'])

# Transform

The Transform component performs feature engineering for both training and serving. 
- It uses the TensorFlow Transform library.
- Transform will take as input the data from ExampleGen, the schema from SchemaGen, as well as a module that contains user-defined Transform code.

Note: The %%writefile cell magic will save the contents of the cell as a .py file on disk. This allows the Transform component to load your code as a module.

In [18]:
_vitals_constants_module_file = '../code/vitals_constants.py'

In [19]:
%%writefile {_vitals_constants_module_file}

NUMERICAL_FEATURES = ['HR', 'Resp', 'Temp']

# Keys
LABEL_KEY = 'isSepsis'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Writing ../code/vitals_constants.py


## Data Preprocessing function

Next, we write a preprocessing_fn that takes in raw data as input, and returns transformed features that our model can train on:

In [20]:
_vitals_transform_module_file = '../code/vitals_transform.py'

In [21]:
%%writefile {_vitals_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
import importlib
importlib.reload(vitals_constants)

_NUMERICAL_FEATURES = vitals_constants.NUMERICAL_FEATURES
_LABEL_KEY = vitals_constants.LABEL_KEY


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[vitals_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  outputs[_LABEL_KEY] = _fill_in_missing(inputs[_LABEL_KEY]) 
    
  return outputs

Writing ../code/vitals_transform.py


## Transform data with Preprocessing func.
Now, we pass in this feature engineering code to the Transform component and run it to transform your data.

In [22]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_vitals_transform_module_file))
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-prototype/code/vitals_transform.py' (including modules: ['vitals_constants', 'vitals_transform']).
INFO:absl:User module package has hash fingerprint version a78414e6795080ead7db7a0850fcfbaea824e392c9ef706dea179dfe0ad5a918.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmp9_f81ivs/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmphr6w_dbf', '--dist-dir', '/tmp/tmpf47_id4e']
INFO:absl:Successfully built user code wheel distribution at '../pipeline/_wheels/tfx_user_code_Transform-0.0+a78414e6795080ead7db7a0850fcfbaea824e392c9ef706dea179dfe0ad5a918-py3-none-any.whl'; target user module is 'vitals_transform'.
INFO:absl:Full user module path is 'vitals_transform@../pipeline/_wheels/tfx_user_code_Transform-0.0+a78414e6795080ead7db7a0850fcfbaea824e392c9ef706dea179dfe0ad5a918-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:ab

Instructions for updating:
Use ref() instead.


INFO:absl:Feature DBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature EtCO2 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature MAP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature O2Sat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Resp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature SBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature isSepsis has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature patient_id has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature record_date has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature record_time has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature EtCO2 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no

INFO:tensorflow:Assets written to: ../pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/e9a6908141ef43978c92fca766c3af64/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:Assets written to: ../pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/b657dae1e77d4e548ddac35b5c4bd6a3/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Examine Transform artifacts. 
This component produces two types of outputs:
1. transform_graph is the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
1. transformed_examples represents the preprocessed training and evaluation data.

In [23]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}),
 'post_transform_schema': OutputChannel(arti

### Examine the Transform graph artifacts.

1. The transformed_metadata subdirectory contains the schema of the preprocessed data. 
1. The transform_fn subdirectory contains the actual preprocessing graph. 
1. The metadata subdirectory contains the schema of the original data.

In [24]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

### Examine some transformed data.

In [25]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "HR_xf"
    value {
      float_list {
        value: -0.6613534092903137
      }
    }
  }
  feature {
    key: "Resp_xf"
    value {
      float_list {
        value: -0.1939026415348053
      }
    }
  }
  feature {
    key: "Temp_xf"
    value {
      float_list {
        value: 2.2171924114227295
      }
    }
  }
  feature {
    key: "isSepsis"
    value {
      int64_list {
        value: 0
      }
    }
  }
}



After the Transform component has transformed your data into features, and the next step is to train a model.

# Trainer

The Trainer component will train a model that you define in TensorFlow. 
- Default Trainer support Estimator API, to use Keras API, you need to specify Generic Trainer by setup custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor) in Trainer's contructor.
- Trainer takes as input the schema from SchemaGen, the transformed data and graph from Transform, training parameters, as well as a module that contains user-defined model code.

In [26]:
_vitals_trainer_module_file = '../code/vitals_trainer.py'

In [27]:
%%writefile {_vitals_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
import importlib
importlib.reload(vitals_constants)

_LABEL_KEY = vitals_constants.LABEL_KEY

_BATCH_SIZE = 40


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying vitals data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)
  
  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)


Writing ../code/vitals_trainer.py


## Train the model.

Now, we pass in this model code to the Trainer component and run it to train the model.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_vitals_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-prototype/code/vitals_trainer.py' (including modules: ['vitals_trainer', 'vitals_constants', 'vitals_transform']).
INFO:absl:User module package has hash fingerprint version deccc9d4e9cb22fe04bf4bbb6946187a90c1443c516360dc3bf342a1393d114f.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpe1b6rc7j/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpk74jxq32', '--dist-dir', '/tmp/tmpnrdx_o5m']
INFO:absl:Successfully built user code wheel distribution at '../pipeline/_wheels/tfx_user_code_Trainer-0.0+deccc9d4e9cb22fe04bf4bbb6946187a90c1443c516360dc3bf342a1393d114f-py3-none-any.whl'; target user module is 'vitals_trainer'.
INFO:absl:Full user module path is 'vitals_trainer@../pipeline/_wheels/tfx_user_code_Trainer-0.0+deccc9d4e9cb22fe04bf4bbb6946187a90c1443c516360dc3bf342a1393d114f-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
I

## Analyze model with Tensorboard

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

In [ ]:
# TODO Tensorboard timesout and doesnt load

model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard
# The %tensorboard magic has exactly the same format as the TensorBoard command line invocation, but with a %-sign in front of it.
%tensorboard --logdir {model_run_artifact_dir}

# Evaluator

The Evaluator component computes model performance metrics over the evaluation set. 
- It uses the TensorFlow Model Analysis library. 
- The Evaluator can also optionally validate that a newly trained model is better than the previous model. 
- This is useful in a production pipeline setting where you may automatically train and validate a model every day.
- In this notebook, we only train one model, so the Evaluator automatically will label the model as "good".

Evaluator will take as input the data from ExampleGen, the trained model from Trainer, and slicing configuration.

In [ ]:
# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import sys
sys.path.append('../code/')
import vitals_constants
import importlib
importlib.reload(vitals_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=vitals_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column HR.
        tfma.SlicingSpec(
            feature_keys=['HR'])
    ])

## Run Evaluator
Next, we give this configuration to Evaluator and run it.

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

## Examine Evaluator artifacts.

Now let's examine the output artifacts of Evaluator.

In [ ]:
evaluator.outputs

Using the evaluation output we can show the default visualization of global metrics on the entire evaluation set.

In [ ]:
context.show(evaluator.outputs['evaluation'])

To see the visualization for sliced evaluation metrics, we can directly call the TensorFlow Model Analysis library.

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column HR.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='HR')

This visualization shows the same metrics, but computed at every feature value of HR instead of on the entire evaluation set.

TensorFlow Model Analysis supports many other visualizations, such as Fairness Indicators and plotting a time series of model performance. To learn more, see https://www.tensorflow.org/guide/keras/sequential_model

Since we added thresholds to our config, validation output is also available. The precence of a blessing artifact indicates that our model passed validation. Since this is the first validation being performed the candidate is automatically blessed.

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

Now can also verify the success by loading the validation result record:

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

# Pusher

The Pusher component is usually at the end of a TFX pipeline. It checks whether a model has passed validation, and if so, exports the model to _serving_model_dir.

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

## Examine Pusher artifacts.

Let's examine the output artifacts of Pusher.

In [ ]:
pusher.outputs

In particular, the Pusher will export your model in the SavedModel format, which looks like this:

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)